### Question 1: Missing Logs

I just spun up the latest agent on my ubuntu 18.04 box and am trying to collect syslogs. I have set up agent to tail the syslog file with the following setup in my yaml file:

```
logs:
  - type: file
    source: syslog
    service: syslog
path: /var/log/my_syslog/syslog.log
```

I know this setup is 100% correct as this is functioning on another host of mine; however, I am seeing the following in the Datadog agent logs:

```
2019-12-23 21:47:22 UTC | CORE | WARN | (pkg/logs/input/file/file_provider.go:77 in FilesToTail) | Could not collect files: file /var/log/my_syslog/syslog.log does not exist
```

However, I can confirm this file does in fact exist running the following shows it does exist on this server fine:

```
sudo ls -ld /var/log/my_syslog/
d--------- 2 root root 4096 Dec 23 21:46 /var/log/my_syslog/
sudo ls -ltr /var/log/my_syslog/
total 0
---------- 1 root root 0 Dec 23 21:41 syslog.log
```

Can you help me figure out the proper configuration here to get my syslogs into Datadog for this server?

### Answer 1:

#### Troubleshooting

- Based on all information provided, there are following hypothetis

    - Collecting logs is disable by default in `datadog-agent`. You have to check if you could see `logs_enabled: true` in your `datadog.yaml`
       - However, since we see there is an information `pkg/logs/input/file/file_provider.go:77 in FilesToTail` - the agent tried to tail the log file. Most likely the log enabled already
       
    - Based on your `ls` command, we can see that the `syslog.log` file is exist. But the file NOT owned by the `dd-agent` and also there is no permssion to read the directory `my_syslog` as well as the file `syslog.log`. This is high potential the root cause `datadog-agent` could not read your log. Refer to this documentation: https://docs.datadoghq.com/agent/troubleshooting/permissions/?tab=agentv63
    

#### Solution

- Change the owner for your directory and file

```bash
sudo chown -R dd-agent:dd-agent /var/log/my_syslog/
```

- Change mode to `644` to let it readable from agent

```bash
sudo chmod -R 644 /var/log/my_syslog/
```

### Question 2: No Traces Visible

I have recently spun up APM in a dockerized environment with both my Python application running in a docker container and the Datadog agent (using the latest image and latest version of the python tracer) running in a docker container. For some reason, I am not seeing any traces in the DD UI. According to the agent logs, no traces are even being received. I am certain the application is receiving traffic (this is in our production environment). I use the following to launch the agent container:

```bash
docker run -d -v /var/run/docker.sock:/var/run/docker.sock:ro \ 
              -v /proc/:/host/proc/:ro \
              -v /sys/fs/cgroup/:/host/sys/fs/cgroup:ro \
              -p 127.0.0.1:8126:8126/tcp \
              -e DD_API_KEY=<API_KEY_REDACTED> \
              -e DD_APM_ENABLED=true \
              datadog/agent:latest
```

And then I use this to launch my python app once the Docker container is up and running:

```bash
ddtrace-run python web-store.py
```

Please let me know how to get traces to report.

### Answer 2:

- In order to answer this question (or troubleshoot) this issue more accurately. There are following questions need to answer:
    - You mentioned your Python application running in a Docker container as well as your Datadog agent. So the command `ddtrace-run python web-store.py` was run inside your Python container?
    
    - If yes, since the two container (Python application and Datadog) are not in the same network. So of course, the log of your `web-store.py` cannot come to the agent.
    
    - The solution to trac3 from other containers:
        + Using `DogStatsD`
        + Traces can be submitted to the Agent from other containers either using Docker networks or with the Docker host IP



#### Solution: Traces from other container

1. Using Docker networks

```bash
# Datadog Agent
docker run -d --name datadog-agent \
      --network <NETWORK_NAME> \
      -v /var/run/docker.sock:/var/run/docker.sock:ro \
      -v /proc/:/host/proc/:ro \
      -v /sys/fs/cgroup/:/host/sys/fs/cgroup:ro \
      -e DD_API_KEY="<DATADOG_API_KEY>" \
      -e DD_APM_ENABLED=true \
      -e DD_APM_NON_LOCAL_TRAFFIC=true \
      datadog/agent:latest

# Application
docker run -d --name app \
      --network <NETWORK_NAME> \
      company/app:latest
```

 - Change your `web-store.py` by adding this snippet at the very begining

```python
from ddtrace import tracer

tracer.configure(
    hostname='datadog-agent',
    port=8126,
)
```

2. Using docker host IP

```bash
docker exec -it datadog-agent bash

# get Datadog container IP, for example: 172.17.0.1
ifconfig
```

- Change your `web-store.py` by adding this snippet at the very begining
```python
from ddtrace import tracer

tracer.configure(
    hostname='172.17.0.1',
    port=8126,
)
```


### Question 3: No custom metrics for container agent

I’m migrating to Docker and recently upgraded our agents to use the latest version of the containerized agent. Everything is working great except for our custom metric submission.
I’m using python library for Dogstatsd to submit a simple metric from another docker container to the agent container (which is set to leverage the host network) but I’m getting none of my Dogstatsd custom metrics. The script has not been changed at all and is sending gauge type metrics over UDP. Our script is as follows:

```python
from datadog import initialize, statsd from hit_rate.py import page_hits_num

import os
import time
options = {
   'statsd_host':'127.0.0.1',
   'statsd_port':8125
}
initialize(**options)
tags=["app:"+os.environ["appname"],"env:"+os.environ["environment"]
]
statsd.gauge('page.hits', page_hits_num, tags=tags)
```

### Answer 3: https://docs.datadoghq.com/developers/dogstatsd/unix_socket/?tab=docker

If you use `DogStatsD` with the Container Agent you must instantiate the host to which `StatsD` metrics are forwarded to with the `DD_DOGSTATSD_SOCKET` environment variable if using a Unix Domain Socket, or with the `DD_AGENT_HOST` environment variable if you are using the host port binding method

- The question for you is what option you choose when you run your `Docker`?


- To set up `DogStatsD` with Unix Domain Socket, enable the `DogStatsD` server through the `dogstatsd_socket` parameter. Then, configure the `DogStatsD` client in your code

    + 1. Set the socket path with the `DD_DOGSTATSD_SOCKET=<YOUR_UDS_PATH>` environment variable on the Agent container.

    + 2. Make the socket file accessible to the client containers by mounting a host directory on both sides (read-only in your client containers and read-write in the Agent container). Mounting the parent folder instead of the individual socket enables socket communication to persist across DogStatsD restarts:

         Start the Agent container with -v /var/run/datadog:/var/run/datadog
       
         Start your application containers with -v /var/run/datadog:/var/run/datadog:ro


- To send metrics from shell scripts, or to test that DogStatsD is listening on the socket, you can use netcat. Most implementations of netcat (ex. netcat-openbsd on Debian or nmap-ncat on RHEL) support Unix Socket traffic via the -U flag:

```bash
echo -n "custom.metric.name:1|c" | nc -U -u -w1 /var/run/datadog/dsd.socket
```

### Question 4: Logs not getting parsed

The sample above raw log looks like this:
MESSAGE: 

```
'{'date': 1577142305298,"message_RDB": "10 MB of memory used by copy-on-write", "pid": 12336, "redis":{"severity": "notice"}, "role": "C", "severity": "*"}'
```

Can you assist and let me know why these JSON objects aren’t parsed properly?

### Answer:

- We can easily see that the `MESSAGE` is malformed JSON. The correct one should be

```json
{"date": 1577142305298, "message_RDB": "10 MB of memory used by copy-on-write", "pid": 12336, "redis": {"severity": "notice"}, "role": "C", "severity": "*"}
```

### Question 5: App Analytics is blank

I have set up APM on my Node application running on an AWS EC2 instance. I can see traces coming into the Services UI but I’m not seeing anything come through in App
Analytics. What gives? I set up my node app to use the latest version of the tracer via npm and import the tracer at my application’s entry point via:

```node
const tracer = require('dd-trace').init()
```

Please let me know how to get these analyzed spans to appear. Note I am using the latest version 7 of the agent.

### Answer 5:

- Look likely the Trace and Span IDs are note injected automatically in your setup

- To enable injection with the environment variable `DD_LOGS_INJECTION=true` or by configuring the tracer directly:

```node
const tracer = require('dd-trace').init({
    logInjection: true
});
```

- Bear in mind that, the automatic injection only works for logs formatted as JSON

- You need also restart the `datadog-agent` to apply new configuration



### Question 6: Synthetic tests failing

I have set up your synthetics product api tests to hit a handful of endpoints for our e- commerce website from around the globe. The problem is, I know these endpoints are up and running as I’ve sshed into a few boxes from the failing regions (note they’re all in AWS
like the synthetic tests) and curl the endpoints successfully (2XX codes all around.) I still have Pingdom running too and everything looks green for these 4 or 5 endpoints. The tests are nothing fancy (mostly GET requests) with assertions around expected status codes and some latency tracking. Instead I’m getting a number of 400/500 codes here with a handful of 403’s even for one region. Please let us know if this is a bug on your end and/or how I can adjust the tests to function.

### Answer 6:

-  First of all, in order to understand what was the root cause of these fail. We need to understand the behavior of your API


- Let look at the meaning of each `http status code`:


    + 400: Bad Request -  response status code indicates that the server cannot or will not process the request due to something that is perceived to be a client error (e.g., malformed request syntax, invalid request message framing, or deceptive request routing).
    
    + 500: Internal Server Error - server error response code indicates that the server encountered an unexpected condition that prevented it from fulfilling the request.
    
    + 403: Forbidden - client error status response code indicates that the server understood the request but refuses to authorize it

- So that, as your test, we can ensure that your API is up and running and possibly you haven't configured the API `auth` when you create the `test`
    
    + adding an API key to header by using `x-api-key=<YOUR KEY>`
    
    + using any `auth` method that you setup for your API
    
    
- There is also possiblity that there is might an unexpected exception that you didn't handle in your backend


- So in order to suggest more accurated solution to fix the issue, we will need to see the detail of each request you made (or setup in the Synthetic test)